In [138]:
%%capture
%load_ext autoreload
%autoreload 2
%pip install beautifulsoup4 requests_html
import requests
from requests_html import HTMLSession
import json
import re
from bs4 import BeautifulSoup as bs
from SteamUser import *
import concurrent.futures
import time

In [184]:
all_steam_games = "https://steamcommunity.com/profiles/{}/games/?tab=all"

session = HTMLSession()

In [176]:
r = session.get(all_steam_games.format('76561197993345099'))

games = re.findall(r'rgGames = .*;', r.text)

games = games[0]

games = ['{"appid"' + g[:-1] for g in re.split('{"appid"', re.split('\]|\[', games)[1]) if len(g) > 0]

if games[-1][-1].isalpha(): games[-1] = games[-1] + '"'
games[-1] = games[-1] + '}'

for g in games: print(g, end='\n\n\n')

games = [json.loads(g) for g in games]


ugames = [g['appid'] for g in games]
ugame_count = len(games)
uplaytime = [(g['appid'], float(g['hours_forever'].replace(',', ''))) for g in games if g.get('hours_forever') != None]
urecent_playtime = [(g['appid'], float(g['hours'].replace(',', ''))) for g in games if g.get('hours') != None]
utotal_playtime = sum([float(g['hours_forever'].replace(',', '')) for g in games if g.get('hours_forever') != None])
ulast_played = [(g['appid'], g['last_played']) for g in games if g.get('last_played') != None]

print("Games : " + str(ugames))
print("Game Count : " + str(ugame_count))
print("Games Playtime : " + str(uplaytime))
print("Games Recent Playtime : " + str(urecent_playtime))
print("Games Total Playtime: " + str(utotal_playtime))
print("Games Last Played: " + str(ulast_played))

{"appid":570,"name":"Dota 2","is_visible_in_steam_china":1,"app_type":1,"logo":"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/570\/capsule_184x69.jpg","friendlyURL":false,"availStatLinks":{"achievements":false,"global_achievements":false,"stats":false,"gcpd":true,"leaderboards":false,"global_leaderboards":false},"hours_forever":"1,429","last_played":1616943789}


{"appid":440,"name":"Team Fortress 2","has_adult_content":1,"app_type":1,"logo":"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/440\/capsule_184x69.jpg","friendlyURL":"TF2","availStatLinks":{"achievements":true,"global_achievements":true,"stats":true,"gcpd":true,"leaderboards":true,"global_leaderboards":true},"hours_forever":"756","last_played":1405295068}


{"appid":1364390,"name":"Animaze","app_type":2,"logo":"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/1364390\/capsule_184x69.jpg","friendlyURL":false,"availStatLinks":{"achievements":false,"global_achievements":false,"stats":false,"gcpd":false,"le

In [177]:
def get_users_games(u):
    r = session.get(all_steam_games.format(u.steamID))

    games = re.findall(r'rgGames = .*;', r.text)

    if games != ['rgGames = [];'] and games != []:
        games = games[0]

        games = ['{"appid"' + g[:-1] for g in re.split('{"appid"', re.split('\]|\[', games)[1]) if len(g) > 0]
        if games[-1][-1].isalpha(): games[-1] = games[-1] + '"'
        games[-1] = games[-1] + '}'

        try:
            games = [json.loads(g) for g in games]
        except json.JSONDecodeError:
            print("Error with user!!! {}".format(u))

        u.games = [g['appid'] for g in games]
        u.game_count = len(games)
        u.playtime = [(g['appid'], float(g['hours_forever'].replace(',', ''))) for g in games if g.get('hours_forever') != None]
        u.recent_playtime = [(g['appid'], float(g['hours'].replace(',', ''))) for g in games if g.get('hours') != None]
        u.total_playtime = sum([float(g['hours_forever'].replace(',', '')) for g in games if g.get('hours_forever') != None])
        u.last_played = [(g['appid'], g['last_played']) for g in games if g.get('last_played') != None]

    time.sleep(1)
    
    return u

In [178]:
users = {}
userslist = []
with open("users22k.json", "r") as file:
    for l in file.readlines():
        u = SteamUser.fromdict(json.loads(l))
        users[u.steamID] = u
    file.close()

len(users)

22130

In [179]:
uz = list(users.values())
sum(u.games == None for u in uz[:100])

100

In [185]:
exe = concurrent.futures.ThreadPoolExecutor(max_workers=1)
f = list(exe.map(get_users_games, uz[:100]))


In [186]:
sum(u.games == None for u in f)

89

In [137]:
import random
random.choice([u for u in users.values() if u.games == None])

SteamUser(steamID='76561197970323416', personaname='TomB', profileurl='https://steamcommunity.com/id/tomqbui/', avatar='https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/44/44b73d1999bbe6311426b8202a66121f7e0734dd.jpg', avatarmedium='https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/44/44b73d1999bbe6311426b8202a66121f7e0734dd_medium.jpg', avatarfull='https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/44/44b73d1999bbe6311426b8202a66121f7e0734dd_full.jpg', personstate=None, communityvisibilitystate=3, profilestate=1, lastlogoff=None, commentpermission=None, realname='Tom Bui', primaryclanid='103582791429521412', timecreated=1100586952, gameid=None, cityid=None, loccountrycode=None, locstatecode=None, loccityid=None, friends=['76561197960265738', '76561197960265740', '76561197960265749', '76561197960269040', '76561197960276281', '76561197960279927', '76561197960402283', '76561197960405535', '76561197960423941', '76561197960434622', '